In [1]:
from transformers import AutoTokenizer, AutoModel

model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer.decode(idx) for idx in token_ids])



/home/prych/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [44]:
import random
#lines = open('data/reviews_for_task3.txt').readlines()

def representation(L):
    txt = ' '.join(L)
    input_ids = tokenizer(txt, return_tensors='pt')['input_ids'].to(device)
    output = model(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

def spoil(L):
    res = []
    for w in L:
        if random.random() < 0.85:
            res.append(w)
        else:
            res.append(w.upper())
    return res

lines = []

for x in open('data/dobre.txt'):
    lines.append('GOOD ' + x.strip())
for x in open('data/zle.txt'):
    lines.append('BAD ' + x.strip())
    
random.shuffle(lines)

N = len(lines)
test_size = N // 4
train_size = N - test_size

train_lines = lines[:train_size]
test_lines = lines[train_size:]

X_train = []
y_train = []
X_test = []
y_test = []

for line in train_lines:
    L = line.split()
    y = 0 if L[0] == 'BAD' else 1
    
    x = representation(L[1:])
    y_train.append(y)
    X_train.append(x)
    
    for i in range(3):
        x = representation(spoil(L[1:]))
        y_train.append(y)
        X_train.append(x)
    
    if len(X_train) % 100 == 0:
        print (len(X_train))

for line in test_lines:
    L = line.split()
    y = 0 if L[0] == 'BAD' else 1
    
    x = representation(L[1:])
    y_test.append(y)
    X_test.append(x)
        
    if len(X_test) % 100 == 0:
        print (len(X_test))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
100
200
300
400
500
600
700
800
900


In [45]:
N = len(lines)
test_size = N // 4
train_size = N - test_size




In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm 

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

#Train accuracy: 0.9348939283101683
#Test accuracy: 0.8715697036223929

Train accuracy: 0.9278529626920263
Test accuracy: 0.8594950603732162


/home/prych/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
